In [1]:
import nltk

In [144]:
class TopDownParser:
    def __init__(self, grammar):
        self.grammar = grammar  # The CFG grammar from NLTK
        self.start_symbol = grammar.start()  # Typically 'S'
        self.tokens = []
        self.index = 0  # Track current token index
        self.backtrack = []  # Stack to store backtracking points
        self.isBacktrack = False

    def parse(self, tokens):
        """
        Parse the tokens using recursive descent with backtracking.
        :param tokens: The list of tokens to parse (input sentence).
        :return: Parse tree if successful, None otherwise.
        """
        self.tokens = tokens
        self.index = 0  # Reset token index for new parsing attempt
        self.prev_index = 0
        self.backtrack = []
        self.prev_tree = []
        self.isBacktrack = False

        parse_tree = self._parse_symbol(self.start_symbol)
        
        if parse_tree and self.index == len(tokens):
            return parse_tree
        
        if self.backtrack and parse_tree:
            self.isBacktrack = True
            production, original_index = self.backtrack.pop()
            self.index = original_index
            self._parse_production(production.rhs(), parse_tree)
            self.isBacktrack = False
            return parse_tree
        
        return None  # Return None if parsing failed

    def _parse_symbol(self, symbol):
        """
        Recursively attempt to parse based on the current symbol.
        :param symbol: The current grammar symbol (either non-terminal or terminal).
        :return: Nested list representing parse tree if successful, None otherwise.
        """
        if isinstance(symbol, nltk.Nonterminal):  # If non-terminal, expand it
            productions = self.grammar.productions(lhs=symbol)
            back_state_list = []
            for i in range(len(productions)):
                original_index = self.index
                subtree = [symbol]  # Create a subtree for this symbol
                if self._parse_production(productions[i].rhs(), subtree):  # Try expanding with this production
                    back_state_list.append((subtree, self.index))
                else:
                    self.index = original_index  # Backtrack on failure
            if back_state_list:
                if len(back_state_list) > 1:
                    self.backtrack.append(back_state_list[1:])
                    print(self.backtrack)
                return back_state_list[0][0]        
            return None  # No production matched
        else:  # If terminal, try to match it with the current token
            print(symbol, self.tokens[self.index])
            if self.index < len(self.tokens) and symbol == self.tokens[self.index]:
                self.index += 1  # Consume the token
                return symbol
            else:
                return None  # Terminal did not match

    def _parse_production(self, rhs, subtree):
        """
        Recursively parse each symbol in the right-hand side of a production.
        :param rhs: The right-hand side symbols of a production.
        :param subtree: The current subtree being built for the production.
        :return: True if successfully matches all symbols in rhs, False otherwise.
        """
        for symbol in rhs:
            result = self._parse_symbol(symbol)  # Recursively parse each symbol
            if result is None:
                return False
            
            # if subtree[0] == nltk.Nonterminal("S"):
            #     self.prev_tree.append(subtree)
                
            subtree.append(result) 

        return True

    def penn_tree_bank_format(self, tree):
        """
        Pretty print the tree in a bracketed format similar to the Penn Treebank format.
        """
        if tree is None:
            return "()"
        
        if isinstance(tree, str):  # Leaf node (terminal)
            return tree
        else:
            symbol = tree[0]
            children = " ".join(self.penn_tree_bank_format(child) for child in tree[1:])
            return f"({symbol} {children})"

In [3]:
from models import Grammar

In [4]:
with open('../output/grammar.txt', 'r') as rules_file:
    rules = rules_file.read() 
    grammar = Grammar(rules=rules)

In [5]:
from models import Tokenizer

In [117]:
tokinizer = Tokenizer(grammar.grammar)
tokens, _ = tokinizer.tokenize("tớ không thử sản phẩm đó")

In [25]:
tokens

['tớ', 'không', 'thử', 'sản phẩm', 'đó']

In [145]:
parser = TopDownParser(grammar.grammar)
tree = parser.parse(tokens)

2
2
2
hiện tại tớ
bây giờ tớ
5
tôi tớ
mình tớ
tớ tớ
anh không
chị không
7
4
đang không
vẫn không
sẽ không
đã không
3
7
có không
muốn không
thích không
biết không
mua không
thử không
quan tâm không
7
có không
muốn không
thích không
biết không
mua không
thử không
quan tâm không
7
có không
muốn không
thích không
biết không
mua không
thử không
quan tâm không
1
7
bận không
hào hứng không
hài lòng không
tốt không
phù hợp không
đắt không
rẻ không
4
đang không
vẫn không
sẽ không
đã không
1
7
có không
muốn không
thích không
biết không
mua không
thử không
quan tâm không
4
đang không
vẫn không
sẽ không
đã không
1
1
không không
7
có thử
muốn thử
thích thử
biết thử
mua thử
thử thử
quan tâm sản phẩm
2
6
nhu cầu sản phẩm
sản phẩm sản phẩm
dịch vụ đó
thông tin đó
cuộc gọi đó
chương trình đó
6
nhu cầu đó
sản phẩm đó
dịch vụ đó
thông tin đó
cuộc gọi đó
chương trình đó
2
1
cái đó
2
6
nhu cầu đó
sản phẩm đó
dịch vụ đó
thông tin đó
cuộc gọi đó
chương trình đó
6
nhu cầu đó
sản phẩm đó
dịch vụ đó
thông tin đ

In [139]:
print(tree)

None
